# Tutorial 6.3. Structural Modeling Using Parametric Optimizable Beam Model

### Description: The structural modeling can be done by simplified MDoF system as seen in excercise 3. Here a robust and generic modeling of MDoF system is presented through a Parametric Optimizable beam. The beam model can be obtained from the github page [ParOptBeam](https://github.com/mpentek/ParOptBeam). This is a guide for setting up an example problem in the framework. The exercise demonstrates some of the capabilities and students are advised to check the implementation and other capabilities. 


Project: Structural Wind Engineering WS20-21
        Chair of Structural Analysis @ TUM - R. Wüchner, M. Péntek, A. Kodakkal
        
Author: anoop.kodakkal@tum.de, mate.pentek@tum.de

Created on:  12.12.2019

Last update: 20.12.2020

##### Contents:

 1. Input parameters
 2. Available beam models - Timoshenko, Euler-Bernoulli etc
 3. Available analysis type - Static, Dynamic, Eigen value analysis etc

### Step 0: Clone the repo
either

#### 1.  'git clone https://github.com/mpentek/ParOptBeam.git '

#### 2.  download as zip file the contents of [ParOptBeam](https://github.com/mpentek/ParOptBeam/archive/master.zip)

#### Important: Run this script from the directory ParOptBeam


importing the required packages and files 

In [1]:
# import
from source.model.structure_model import StraightBeam
from source.analysis.analysis_controller import AnalysisController

The inputs are defined as a dictionaries with various options. The options are as listed below with some basic explanations. Most of the naming are self explanatory. The unit system adopted are SI units(N,m,s) unless defined otherwise. 

__model_parameters__ : defines the details of the model
domain_size can be 2D or 3D 
other options for boundary_conditions 
* fixed-fixed
* pinned-pinned
* fixed-pinned
* pinned-fixed
* fixed-free
* free-fixed

In [2]:
# inputs 
parameters = {}

parameters["model_parameters"] = {
        "name": "GenericBuilding",
        "domain_size": "3D",
        "boundary_conditions": "fixed-free"}


__system_parameters__: defines the beam formulation 

available models are 
* Bernoulli beam
* Timoshenko beam
* Co-rotational beam

available models 
* Linear 
* Non-linear


In [3]:
parameters["model_parameters"]["system_parameters"]={
            "element_params": {
                "type": "Timoshenko",
                "is_nonlinear": False
            }}


__material_parameters__ : defines the material properties of the beam 

you may define the material properties of the beam

In [4]:

parameters["model_parameters"]["system_parameters"]["material"] = {
                "is_nonlinear": False,
                "density": 160.0,
                "youngs_modulus": 2.861e8,
                "poisson_ratio": 0.1,
                "damping_ratio": 0.0
            }

__geometry_parameters__ : defines the geometry of the structure

It is possible to define the various intervals of the structure with varying geometric parameters

number of elements indicate to the total number of elements and not on each intervals 
__coordinate system__
the following coordinate system may be (not necessarily) followed for the project 

x - along the longitudinal axis 

y - along the wind 

z - across the wind

In [5]:
parameters["model_parameters"]["system_parameters"]["geometry"] = {
                "length_x": 180,
                "number_of_elements": 3,
                "defined_on_intervals": [{
                    "interval_bounds" : [0.0, 60.0],
                    "length_y": [55.0],
                    "length_z": [35.0],
                    "area"    : [1925.0],
                    "shear_area_y" : [1605.0],
                    "shear_area_z" : [1605.0],
                    "moment_of_inertia_y" : [196510.0],
                    "moment_of_inertia_z" : [458260.0],
                    "torsional_moment_of_inertia" : [691771.0]},
                    {
                    "interval_bounds" : [60.0, 120.0],
                    "length_y": [45.0],
                    "length_z": [30.0],
                    "area"    : [1350],
                    "shear_area_y" : [1125.0],
                    "shear_area_z" : [1125.0],
                    "moment_of_inertia_y" : [101250.0],
                    "moment_of_inertia_z" : [227813.0],
                    "torsional_moment_of_inertia" : [329063.0]},
                    {
                    "interval_bounds" : [120.0, "End"],
                    "length_y": [35.0],
                    "length_z": [25.0],
                    "area"    : [875.0],
                    "shear_area_y" : [729.0],
                    "shear_area_z" : [729.0],
                    "moment_of_inertia_y" : [45573.0],
                    "moment_of_inertia_z" : [89323.0],
                    "torsional_moment_of_inertia" : [134896.0]}]
            }


__optimization_parameters__: defines the optimization parameters for fine tuning the structure. 

It is possible to do an optimization of various structural parameters to match to eigen frequency for various modes. 
The density can also be optimized to match the total weight of the building. 

In [6]:
parameters["optimization_parameters"] =  {
        "adapt_for_target_values": {
            "density_for_total_mass": 38880000.0,
            "geometric_properties_for": {
                "help": "first entry: sway_y, second entry: sway_z, -1: shear, +1: bending",
                "partition_shear_bending": [-1, 1],
                "consider_decomposed_modes": ["sway_z","sway_y", "torsional"],
                "corresponding_mode_ids" : [1, 1, 1],
                "corresponding_eigenfrequencies": [0.23,0.20,0.40]}}
    }


__analysis_parameters__ : defines the analysis parameters options 

Report directory can be defined as relative/ absolute path

a skin model is also available to visualize the structure


In [7]:
parameters["analyses_parameters"] = {
        "global_output_folder" : "some/path",
        "model_properties": {
            "write": True,
            "plot":True
        }}
parameters["analyses_parameters"]["report_options"] = {
            "combine_plots_into_pdf" : True,
            "display_plots_on_screen" : False,
            "use_skin_model" : False
        }

__skin_model_parameters__ : defines the details of the skin model 

you may check out the other options available


In [8]:
parameters["analyses_parameters"]["skin_model_parameters"] = {
            "geometry": [ [0, -22.5, -15.0], [0, -22.5, 15], [0, 22.5, 15],
                          [0, 22.5, -15]
            ],
            "contour_density": 1,
            "record_animation": True,
            "visualize_line_structure": True,
            "beam_direction": "x",
            "scaling_vector": [1.0, 1.0 , 1.0, 1.0, 1.0, 1.0, 1.0],
            "eigenmode_scaling_factor" : 1e5,
            "dynamic_scaling_factor" : 1e3
        }


__Eigenvalue analysis__ : defines the options for eigen value analysis 

you may check out the other options available

In [9]:

parameters["analyses_parameters"]["runs"] = [{
                "type": "eigenvalue_analysis",
                "settings": {
                    "normalization": "mass_normalized"},
                "input":{},
                "output":{
                    "eigenmode_summary": {
                        "write" : True, 
                        "plot" : True},
                    "eigenmode_identification": {
                        "write" : True, 
                        "plot" : True},
                    "selected_eigenmode": {
                        "plot_mode": [1,2,3], 
                        "write_mode": [1,2,3],
                        "animate_mode": [1],
                        "animate_skin_model": []},
                    "selected_eigenmode_range": {
                        "help": "maximum 4 modes per range at a time",
                        "considered_ranges": [[1,2]], 
                        "plot_range": [True, True], 
                        "write_range": [True, False]}
                    }
            }]


__Dynamic analysis__: defines the options for dynamic analysis 


you may check out the other options available

Note : the force file should match the number of nodes / elements

__Tip: Take extra care on the time integration scheme and time step values__

In [10]:
parameters["analyses_parameters"]["runs"].append(dict({"type" : "dynamic_analysis",
                "settings": {
                    "solver_type": "Linear",
                    "run_in_modal_coordinates": False,
                    "time":{
                        "integration_scheme": "GenAlpha",
                        "start": 0.0,
                        "end": 600.0,
                        "step" : 0.02},
                    "intial_conditions": {
                        "displacement": "None",
                        "velocity": "None",
                        "acceleration" : "None"
                    }},
                "input": {
                    "help":"provide load file in the required format",
                    "file_path": "input/force/generic_building/dynamic_force_4_nodes.npy"
                },
                "output":{
                    "selected_instance": {
                        "plot_step": [1500, 2361], 
                        "write_step": [3276],
                        "plot_time": [30.5, 315.25], 
                        "write_time": [450.15]
                    },
                    "animate_time_history" : False,
                    "animate_skin_model_time_history": False,
                    "skin_model_animation_parameters":{
                        "start_record": 160,
                        "end_record": 200,
                        "record_step": 10
                    },
                    "selected_dof": {
                        "dof_list": [1, 2, 0, 4, 5, 3,
                                    -5, 
                                    -4, 
                                    -2, 
                                    -1],
                        "help": "result type can be a list containing: reaction, ext_force, displacement, velocity, acceleration",
                        "result_type": [["reaction"], ["reaction"], ["reaction"], ["reaction"], ["reaction"], ["reaction"],
                                        ["displacement", "velocity", "acceleration"], 
                                        ["displacement", "velocity", "acceleration"], 
                                        ["displacement", "velocity", "acceleration"], 
                                        ["displacement", "velocity", "acceleration"]],
                        "plot_result": [[True], [True], [True], [True], [True], [True],
                                        [True, True, True], 
                                        [True, True, True], 
                                        [True, False, True], 
                                        [True, False, True]],
                        "write_result": [[False],[False],[False],[True],[True],[True],
                                            [True, False, True], 
                                            [True, False, True], 
                                            [True, False, True], 
                                            [True, False, True]]
                    }
                }
            }))


__Static analysis__: defines the options for static analysis 

you may check out the other options available

Note: the force file should match the number of nodes / elements

In [11]:
parameters["analyses_parameters"]["runs"].append(dict({
                "type" : "static_analysis",
                "settings": {}, 
                "input":{
                    "help":"provide load file in the required format - either some symbolic generated or time step from dynamic",
                    "file_path": "input/force/generic_building/dynamic_force_4_nodes.npy",
                    "is_time_history_file" : True,
                    "selected_time_step" : 15000
                }, 
                "output":{
                    "plot": ["deformation", "forces"],
                    "write": ["deformation"]
                }
            }))


__Creates the beam model and evaluate responses__ 


In [18]:
# create initial model
beam_model = StraightBeam(parameters['model_parameters'])

# additional changes due to optimization
if 'optimization_parameters' in parameters:
    # return the model of the optimizable instance to preserve what is required by analyzis
    from source.model.optimizable_structure_model import OptimizableStraightBeam
    beam_model = OptimizableStraightBeam(
        beam_model, parameters['optimization_parameters']['adapt_for_target_values']).model
else:
    print('No need found for adapting structure for target values')

# ==============================================
# Analysis wrapper

analyses_controller = AnalysisController(
    beam_model, parameters['analyses_parameters'])
analyses_controller.solve()
analyses_controller.postprocess()

No outrigger mass for interval 0
No outrigger mass for interval 1
No outrigger mass for interval 2
3D Timoshenko Beam Element 0
Initial coordinates: 
[0. 0. 0.]
[60.  0.  0.]
A: 1925.0
Asy: 1605.0
Asz: 1605.0
Iy: 196510.0
Iz: 458260.0
Pz: 0.8978650051921081
Py: 2.0938151609553484

3D Timoshenko Beam Element 1
Initial coordinates: 
[60.  0.  0.]
[120.   0.   0.]
A: 1350.0
Asy: 1125.0
Asz: 1125.0
Iy: 101250.0
Iz: 227813.0
Pz: 0.66
Py: 1.4850032592592592

3D Timoshenko Beam Element 2
Initial coordinates: 
[120.   0.   0.]
[180.   0.   0.]
A: 875.0
Asy: 729.0
Asz: 729.0
Iy: 45573.0
Iz: 89323.0
Pz: 0.45843895747599456
Py: 0.8985395518975767

parameters does not have "elastic_fixity_dofs"
[9240000.0, 15720000.0, 10680000.0, 4200000.0]
[9240000.0, 15720000.0, 10680000.0, 4200000.0]
BEFORE OPTIMIZATION
Result of decoupled eigenmode identification for the first 15 mode(s)
  Mode type: sway_y
    Eigenform 1 with eigenfrequency 0.30 Hz
    Eigenform 3 with eigenfrequency 1.22 Hz
    Eigenform 7 

Exception: beam model and dynamic load signal have different number of nodes